In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch

from torch.autograd import Variable
import torch.backends.cudnn as cudnn

from torchvision.utils import save_image

import numpy as np

import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms, models, datasets

In [95]:
import glob

In [96]:
os.path.exists("./output/000")

True

In [97]:
for epoch in range (200):
  if (os.path.exists("./output/000")) == False:
    os.mkdir("./output/%03d" % epoch)
  else:
    files = glob.glob("./output/%03d/*.png" % epoch)

    for f in files:
      os.remove(f)

# Data

In [98]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


def get_backgrounds():
    backgrounds = []
    for file in os.listdir("./images/train"):
        if file.endswith('.jpg'):
            backgrounds.append(plt.imread(os.path.join("./images/train",file)))
    return np.array(backgrounds)
backgrounds = get_backgrounds()


def compose_image(image):
    image = (image > 0).astype(np.float32)
    image = image.reshape([28,28])*255.0
    
    image = np.stack([image,image,image],axis=2)
    
    background = np.random.choice(backgrounds)
    w,h,_ = background.shape
    dw, dh,_ = image.shape
    x = np.random.randint(0,w-dw)
    y = np.random.randint(0,h-dh)
    
    temp = background[x:x+dw, y:y+dh]
    return np.abs(temp-image).astype(np.uint8)


class MNISTM(Dataset):
            
    def __init__(self, train=True,transform=None):
        if train:
            self.data = datasets.MNIST(root='.data/mnist',train=True, download=True)
        else:
            self.data = datasets.MNIST(root='.data/mnist',train=False, download=True)
        self.backgrounds = get_backgrounds()
        self.transform = transform
        self.images = []
        self.targets = []
        for index in range(len(self.data)):
            image = np.array(self.data.__getitem__(index)[0])
            target = self.data.__getitem__(index)[1]
            image = compose_image(image)
            if self.transform is not None:
                image = self.transform(image)
            self.images.append(image)
            self.targets.append(target)
        
    def __getitem__(self,index):
        
        #image = Image.fromarray(image.squeeze(), mode="RGB")
        image = self.images[index]
        target = self.targets[index]
        
        return image, target
        
    def __len__(self):
        return len(self.data)
    
def get_mnistm_loaders(data_aug = False, batch_size=128,test_batch_size=1000):
    if data_aug:
        train_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(64),
            transforms.RandomCrop(64,padding=4),
            transforms.ToTensor()
        ])
    else:
        train_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(64),
            transforms.ToTensor()
        ])
    test_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(64),
        transforms.ToTensor()
    ])
    
    kwargs = {}
    train_loader = DataLoader(
        MNISTM(train=True,transform=train_transform),batch_size=batch_size, shuffle=True, drop_last=True)
    train_eval_loader = DataLoader(
        MNISTM(train=True, transform=test_transform),batch_size=test_batch_size, shuffle=False, drop_last=True)
    test_loader = DataLoader(
        MNISTM(train=False,transform=test_transform),batch_size=test_batch_size, shuffle=False, drop_last=True)
    return train_loader, train_eval_loader, test_loader


def get_mnist_loaders(data_aug = False, batch_size=128,test_batch_size=1000):
    if data_aug:
        train_transform = transforms.Compose(
            [transforms.Resize(64),
            transforms.RandomCrop(64,padding=4),
            transforms.Grayscale(3),
            transforms.ToTensor()
        ])
    else:
        train_transform = transforms.Compose([
            transforms.Resize(64),
            transforms.Grayscale(3),
            transforms.ToTensor()
        ])
    test_transform = transforms.Compose([
        transforms.Resize(64),
        transforms.Grayscale(3),
        transforms.ToTensor()
    ])
    kwargs = {}

    train_loader = DataLoader(
        datasets.MNIST(root='.data/mnist',train=True, download=True,transform=train_transform),batch_size=batch_size, shuffle=True, drop_last=True)
    train_eval_loader = DataLoader(
        datasets.MNIST(root='.data/mnist',train=True, download=True, transform=test_transform),batch_size=test_batch_size, shuffle=False, drop_last=True)
    test_loader = DataLoader(
        datasets.MNIST(root='.data/mnist',train=False, download=True, transform=test_transform),batch_size=test_batch_size, shuffle=False, drop_last=True)
    return train_loader, train_eval_loader, test_loader

C:\Users\55366\AppData\Local\Temp\ipykernel_56604\4071975017.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(backgrounds)


In [99]:
loader_source, mnist_eval_loader, mnist_test_loader = get_mnist_loaders(batch_size=16)
loader_target, mnistm_eval_loader,mnistm_test_loader = get_mnistm_loaders(batch_size=16)

C:\Users\55366\AppData\Local\Temp\ipykernel_56604\4071975017.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(backgrounds)


# Module

## Encoder

In [2]:
class Encoder(nn.Module):
    def __init__(self, input_nc=3):
        super(Encoder, self).__init__()
        self.model = nn.Sequential(
                nn.ReflectionPad2d(3),
                nn.Conv2d(input_nc, 64, 7),
                nn.InstanceNorm2d(64),
                nn.ReLU(inplace=True),

                nn.Conv2d(64, 128, 3, stride=2, padding=1),
                nn.InstanceNorm2d(128),
                nn.ReLU(inplace=True),

                nn.Conv2d(128, 256, 3, stride=2, padding=1),
                nn.InstanceNorm2d(256),
                nn.ReLU(inplace=True),

                nn.Conv2d(256, 512, 3, stride=2, padding=1),
                nn.InstanceNorm2d(512),
                nn.ReLU(inplace=True),

                nn.Conv2d(512, 1024, 3, stride=2, padding=1),
                nn.InstanceNorm2d(1024),
                nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.model(x)


## Decoder

In [14]:
class Decoder(nn.Module):
    def __init__(self, input_nc=1024, output_nc=3):
        super(Decoder, self).__init__()

        self.model = nn.Sequential(
            nn.ConvTranspose2d(input_nc, 512, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(512),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(512, 256, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),

            nn.ReflectionPad2d(3),
            nn.Conv2d(64, output_nc, 7),
            nn.InstanceNorm2d(3),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

## Identity Generator

In [4]:
class Identity_Generator(nn.Module):
    def __init__(self, encoder, decoder):
        super(Identity_Generator, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, A, B):
        latentA = self.encoder(A)
        latentB = self.encoder(B)

        reconstructedA = self.decoder(latentA)
        reconstructedB = self.decoder(latentB)
        return reconstructedA, reconstructedB

## Perceptual

In [27]:
class Perceptual(nn.Module):
    def __init__(self):
        super(Perceptual, self).__init__()

        self.encoder = nn.Sequential(
                nn.ReflectionPad2d(3),
                nn.Conv2d(3, 64, 7),
                nn.InstanceNorm2d(64),
                nn.ReLU(inplace=True),

                nn.Conv2d(64, 128, 3, stride=2, padding=1),
                nn.InstanceNorm2d(128),
                nn.ReLU(inplace=True),

                nn.Conv2d(128, 256, 3, stride=2, padding=1),
                nn.InstanceNorm2d(256),
                nn.ReLU(inplace=True),

                nn.Conv2d(256, 512, 3, stride=2, padding=1),
                nn.InstanceNorm2d(512),
                nn.ReLU(inplace=True),

                nn.Conv2d(512, 1024, 3, stride=2, padding=1),
                nn.InstanceNorm2d(1024),
                nn.ReLU(inplace=True)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(512),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(512, 256, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),

            nn.ReflectionPad2d(3),
            nn.Conv2d(64, 3, 7),
            nn.InstanceNorm2d(3),
            nn.Tanh()
        )

    def forward(self, A, B):
        encode_A = self.encoder(A)
        encode_B = self.encoder(B)
        reconstructedA = self.decoder(encode_A)
        reconstructedB = self.decoder(encode_B)

        encode_A.detach()
        encode_B.detach()

        style = encode_A[:, 0:512, : , :]
        content = encode_B[:, 512:1024, :, :]
        
        mixed_latent = torch.cat([style, content], dim=1)
        mixed_image = self.decoder(mixed_latent)

        return mixed_image, reconstructedA, reconstructedB

## Discriminator

In [6]:
class Discriminator(nn.Module):
    def __init__(self, input_nc = 3):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(input_nc, 64, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, padding=1),
            nn.InstanceNorm2d(128), 
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            nn.InstanceNorm2d(256), 
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, stride=2, padding=1),
            nn.InstanceNorm2d(256), 
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, padding=1)
        )


    def forward(self, x):
        x =  self.model(x)
        return F.avg_pool2d(x, x.size()[2:]).view(x.size()[0], -1)

## Model test

In [28]:
test_tensor_source = torch.rand(20, 3, 64, 64)
test_tensor_target = torch.rand(20, 3, 64, 64)

perceptual = Perceptual()
discriminator = Discriminator()

In [29]:
mix, output_a, output_b = perceptual(test_tensor_source, test_tensor_target)

print(mix.size())
print(output_a.size())
print(output_b.size())

torch.Size([20, 3, 64, 64])
torch.Size([20, 3, 64, 64])
torch.Size([20, 3, 64, 64])


In [30]:
output = discriminator(test_tensor_source)
print(output.size())

torch.Size([20, 1])


In [31]:
print(mix[0][0][0] - output_a[0][0][0])

tensor([ 0.2579, -0.2416,  0.0964,  1.2266,  1.3237,  0.6352, -0.3108,  0.9131,
        -0.7289, -0.4677,  0.5602,  0.3801, -0.8635,  0.1178,  0.4716,  0.4965,
         0.2412, -0.7411, -1.1922, -0.6741,  0.0059, -0.3301, -0.3498, -0.0460,
         0.7104,  0.5532,  0.0353,  0.2016, -0.3188,  0.4341,  0.7073, -0.6594,
        -0.9747, -0.5914, -0.7311, -0.3820,  0.4321,  0.0963, -1.2322,  1.1050,
         0.1490,  0.0208,  0.5074, -0.1431, -0.0056, -0.1938, -1.2501,  0.6673,
        -0.8977,  0.4970,  0.5564,  0.3826,  1.0094, -0.1623,  0.8489, -0.5292,
         0.4148, -0.0783, -0.7048,  0.5939,  0.0921,  0.1221,  0.3708, -0.3768],
       grad_fn=<SubBackward0>)


# Loss

In [114]:
def tv_loss(img, tv_weight=5e-2):
    w_variance = torch.sum(torch.pow(img[:,:,:,:-1] - img[:,:,:,1:], 2))
    h_variance = torch.sum(torch.pow(img[:,:,:-1,:] - img[:,:,1:,:], 2))
    loss = tv_weight * (h_variance + w_variance)
    return loss

def total_variation_loss(img, weight=5e-2):
    bs_img, c_img, h_img, w_img = img.size()
    tv_h = torch.pow(img[:, :, 1:, :] - img[:, :, :-1, :], 2).sum()
    tv_w = torch.pow(img[:, :, :, 1:] - img[:, :, :, :-1], 2).sum()
    return weight * (tv_h + tv_w) / (bs_img * c_img * h_img * w_img)

def compute_content_loss(target_feature, content_feature):
    return torch.mean((target_feature - content_feature)**2)

def batch_gram_matrix(img):
    """
    Compute the gram matrix by converting to 2D tensor and doing dot product
    img: (batch, channel/depth, height, width)
    """
    b, d, h, w = img.size()
    img = img.view(b*d, h*w) # fix the dimension. It doesn't make sense to put b=1 when it's not always the case
    gram = torch.mm(img, img.t())
    return gram

style_weights = {'conv1_1': 1.,
                 'conv2_1': 0.75,
                 'conv3_1': 0.2,
                 'conv4_1': 0.2,
                 'conv5_1': 0.2}

def compute_style_loss(style_features, target_features):
    # the style loss
    # initialize the style loss to 0
    style_loss = 0
    style_grams = {layer: batch_gram_matrix(style_features[layer]) for layer in style_features}
    # then add to it for each layer's gram matrix loss
    for layer in style_weights:
        # get the "target" style representation for the layer
        target_feature = target_features[layer]
        target_gram = batch_gram_matrix(target_feature)
        _, d, h, w = target_feature.shape
        # get the "style" style representation
        style_gram = style_grams[layer]
        # the style loss for one layer, weighted appropriately
        layer_style_loss = style_weights[layer] * torch.mean((target_gram - style_gram) ** 2)
        # add to the style loss
        style_loss += layer_style_loss / (d * h * w)

    return style_loss

# Training

## Training Method

In [115]:
def get_features(image, model, layers=None):
    if layers is None:
        layers = {'0': 'conv1_1',
                  '5': 'conv2_1',
                  '10': 'conv3_1',
                  '19': 'conv4_1',
                  '21': 'conv4_2',  ## content representation
                  '28': 'conv5_1'}

    features_arr = []
    features = {}
    x = image
    # model._modules is a dictionary holding each module in the model
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x

    return features

## Model hypermeter

In [117]:
class LambdaLR():
    def __init__(self, n_epochs, offset, decay_start_epoch):
        assert ((n_epochs - decay_start_epoch) > 0), "Decay must start before the training session ends!"
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch

    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch)/(self.n_epochs - self.decay_start_epoch)

In [118]:
perceptual = Perceptual()
discriminator = Discriminator()

In [119]:
learning_rate = 0.002
beta = (0.5, 0.999)

criterion_adv = torch.nn.MSELoss()
criterion_discriminator = torch.nn.MSELoss()
criterion_construct = torch.nn.L1Loss()

optimizer_pre = torch.optim.Adam(perceptual.parameters(), lr=learning_rate, betas=beta)
optimizer_dis = torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=beta)

In [120]:
vgg = models.vgg19(pretrained=True).features
for param in vgg.parameters():
    param.requires_grad_(False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg.to(device)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [121]:
if (torch.cuda.is_available()):
    torch.cuda.manual_seed_all(42)
    cudnn.benchmark = True

    criterion_construct = criterion_construct.cuda()
    criterion_discriminator = criterion_discriminator.cuda()
    criterion_construct = criterion_construct.cuda()

    perceptual.cuda()
    discriminator.cuda()

## Init the network

In [122]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant(m.bias.data, 0.0)

In [123]:
perceptual.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
)

## Training

In [124]:
batchSize = 16
cuda = True

Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

real_label = Variable(Tensor(batchSize).fill_(1.0), requires_grad=False)
fake_label = Variable(Tensor(batchSize).fill_(0.0), requires_grad=False)
real_label = real_label[:, None]
fake_label = fake_label[:, None]


input_A = Tensor(batchSize, 3, 64, 64)
input_B = Tensor(batchSize, 3, 64, 64)

for epoch in range(0, 200):
    i = -1
    for batchA, batchB in zip(loader_target, loader_source):
        i += 1
        real_A = Variable(input_A.copy_(batchA[0]))
        real_B = Variable(input_B.copy_(batchB[0]))

        optimizer_pre.zero_grad()

        mixed_image, reconstructionA, reconstructionB = perceptual(real_A, real_B)

        loss_ss = criterion_construct(reconstructionA, real_A)*30.0
        loss_tt = criterion_construct(reconstructionB, real_B)*30.0

        TV_loss = total_variation_loss(mixed_image)
        pred_fake = discriminator(mixed_image)
        loss_adv = criterion_adv(pred_fake, real_label)

        cuda_mixed_image = mixed_image.clone().requires_grad_(True).cuda()
        cuda_real_A = real_A.clone().requires_grad_(True).cuda()
        cuda_real_B = real_B.clone().requires_grad_(True).cuda()
        style_features = get_features(cuda_real_A, vgg)
        content_features = get_features(cuda_real_B, vgg)
        target_features = get_features(cuda_mixed_image, vgg)

        content_loss = compute_content_loss(target_features['conv4_2'], content_features['conv4_2']) * 0.1
        style_loss = compute_style_loss(style_features, target_features) * 0.05

        preceptual_loss = loss_ss + loss_tt + TV_loss + loss_adv + content_loss + style_loss

        preceptual_loss.backward()
        optimizer_pre.step()

        optimizer_dis.zero_grad()

        pred_real = discriminator(real_A)
        pred_fake = discriminator(mixed_image.detach())

        loss_dis_real = criterion_discriminator(pred_real, real_label)
        loss_dis_fake = criterion_discriminator(pred_real, fake_label)

        discriminator_loss = loss_dis_real + loss_dis_fake
        discriminator_loss.backward()
        optimizer_dis.step()

        if  i % 200 == 0:
            real_A = real_A.data
            real_B = real_B.data
            mixed_image = mixed_image.data 
            reconstructionA = reconstructionA.data
            reconstructionB = reconstructionB.data

            save_image(real_A, './output/%03d/00_%d_A.png' % ( epoch, i))
            save_image(real_B, 'output/%03d/00_%d_B.png' % ( epoch, i))
            save_image(reconstructionA, 'output/%03d/00_%d_reconA.png' % ( epoch, i))
            save_image(reconstructionB, 'output/%03d/00_%d_reconB.png' % ( epoch, i))
            save_image(mixed_image, 'output/%03d/00_%d_Mixed.png' % ( epoch, i))
        
        if i >= 400:
            break

    print ("e: %d, pre_loss: %.2f, D_real: %.2f, D_fake: %.2f" % (epoch, preceptual_loss, loss_dis_real, loss_dis_fake))
    

e: 0, pre_loss: 14.19, D_real: 0.40, D_fake: 0.15
e: 1, pre_loss: 11.74, D_real: 0.16, D_fake: 0.36
e: 2, pre_loss: 12.23, D_real: 0.23, D_fake: 0.27
e: 3, pre_loss: 10.73, D_real: 0.28, D_fake: 0.22
e: 4, pre_loss: 11.89, D_real: 0.24, D_fake: 0.27
e: 5, pre_loss: 14.07, D_real: 0.24, D_fake: 0.26
e: 6, pre_loss: 11.60, D_real: 0.30, D_fake: 0.21
e: 7, pre_loss: 12.44, D_real: 0.39, D_fake: 0.15
e: 8, pre_loss: 10.10, D_real: 0.28, D_fake: 0.22
e: 9, pre_loss: 10.28, D_real: 0.25, D_fake: 0.26
e: 10, pre_loss: 10.21, D_real: 0.18, D_fake: 0.34
e: 11, pre_loss: 10.48, D_real: 0.31, D_fake: 0.20
e: 12, pre_loss: 11.73, D_real: 0.20, D_fake: 0.31
e: 13, pre_loss: 11.95, D_real: 0.17, D_fake: 0.34
e: 14, pre_loss: 11.59, D_real: 0.20, D_fake: 0.30
e: 15, pre_loss: 10.99, D_real: 0.35, D_fake: 0.16
e: 16, pre_loss: 11.67, D_real: 0.23, D_fake: 0.27
e: 17, pre_loss: 10.77, D_real: 0.26, D_fake: 0.24
e: 18, pre_loss: 10.90, D_real: 0.27, D_fake: 0.23
e: 19, pre_loss: 10.36, D_real: 0.25, D_f

In [125]:
torch.save(perceptual, "./model_weight/perceptual")
torch.save(discriminator, "./model_weight/discriminator")